# 8. 텍스트 데이터 XAI 사용법

## 8.1.검증의 목적
- 이번 장에서는 문장 텍스트의 카테고리 예측 판별 작업에 대해 알아보겠습니다.
- 텍스트 데이터는 테이블 데이터와 화상 데이터와 성질이 매우 다르므로
  방법별로 각각의 순서대로 진행하여 텍스트에 대한 설명 가능 AI 방법을 이해합니다.

## 8.2. 라이브러리 준비

In [ ]:
# 라이브러리 인스톨
!pip install -U pip
!pip install torch==1.7.0 fugashi==1.0.5 ipadic==1.0.0 \
    transformers==4.0.0 lime==0.2.0.1 captum==0.3.0 \
    scikit-learn==0.22.1 numpy==1.19.4
!pip install ipywidgets

In [ ]:
# 라이브러리 임포트
import os
import re
import sys

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients
from lime.lime_text import LimeTextExplainer
import numpy as np
import torch
from torch.nn import functional as F
import transformers
from sklearn.model_selection import train_test_split

In [ ]:
# BERT모델의 동작 디바이스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\

# 난수 계열 지정
torch.manual_seed(0)

In [ ]:
# 데이터 다운로드
!rm -f /tmp/livedoor.tar.gz
!curl https://www.rondhuit.com/download/ldcc-20140209.tar.gz -o /tmp/livedoor.tar.gz
!tar zxf /tmp/livedoor.tar.gz -C /tmp

### Livedoor 컴파스를 읽어들이는 클래스

In [ ]:
class LivedoorData(object):
    def __init__(self, data_dir='/tmp/text/'):
        self.data_dir = data_dir

    def _get_texts(self, category):
        category_dir = os.path.join(self.data_dir, category)
        filenames = [
            filename for filename in os.listdir(category_dir)
            if re.search(r'^.*\.txt$', filename)
        ]
        texts = []
        for filename in filenames:
            fpath = os.path.join(category_dir, filename)
            with open(fpath, 'r', encoding='utf-8') as fp:
                fp.readline()
                fp.readline()
                buf = []
                for line in fp:
                    buf.append(line.strip().replace(r'\s', ''))
            texts.append(' '.join(buf))
        return texts

    def get_categories(self):
        categories = [
            category for category in os.listdir(self.data_dir)
            if os.path.isdir(os.path.join(self.data_dir, category))
               and not re.search(r'^\.', category)
        ]
        return categories
    
    def read(self, categories=None):
        all_categories = self.get_categories()
        if categories:
            categories = [
                category for category in categories
                if category in all_categories
            ]
        else:
            categories = all_categories
        
        self.X, self.y = [], []
        for category in categories:
            texts = self._get_texts(category)
            self.X.extend(texts)
            self.y.extend([category] * len(texts))
        
    def get_data(self):
        if not self.X and not self.y:
            self.read()
        return self.X, self.y

### Livedoor 뉴스 데이터 읽어들임
- 이번에는 다음의 3가지 카테고리 판별을 하기위한 모델을 구축합니다.
  - Sports Watch
  - 가전 채널
  - MOVIE ENTER

In [ ]:
# 이번 대상이 되는 뉴스 카테고리
target_categories = ['sports-watch', 'kaden-channel', 'movie-enter']
n_category = len(target_categories)

# 지정한 뉴스 데이터 추출
#  X: 뉴스 텍스트, y: 카테고리명
livedoor = LivedoorData()
livedoor.read(categories=target_categories)
X, y = livedoor.get_data()

# 카테고리 명의 list를 카테고리 번호로 지정
y_indices = np.array([target_categories.index(v) for v in y])

## 8.4. 모델의 학습과 예측

### BERT모델  / tokenizer 읽어 들임
- 판별 모델로 `BertForSequenceClassification` 를 이용합니다.
- 판별 모델의 입력 데이터 가공용으로 tokenizer 를 준비해 둡니다.
- 해당 모델을 처음 읽어들일 때는 데이터를 다운로드합니다.

In [ ]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification \
    .from_pretrained(model_name, num_labels=len(target_categories),
                     output_attentions=True)

In [ ]:
model = model.to(device)

### 학습용 데이터 생성

In [ ]:
# 최대 token id 길이
max_length = 256

tokenized = tokenizer.batch_encode_plus(
    X, padding=True, truncation=True,
    max_length=max_length, return_tensors='pt')

tensor_X = tokenized['input_ids']
tensor_y = torch.tensor(y_indices)
tensor_mask = tokenized['attention_mask']

In [ ]:
X_train, X_valid, y_train, y_valid, mask_train, mask_valid = train_test_split(
    tensor_X, tensor_y, tensor_mask, train_size=0.75, random_state=42
)

### fine tuning

In [ ]:
n_epoch = 3
batch_size = 16

n_batch_train = X_train.shape[0] // batch_size + 1
n_batch_valid = X_valid.shape[0] // batch_size + 1

optimizer = transformers.AdamW(model.parameters(), lr=1e-5)

In [ ]:
def calculate_loss(model, X_target, y_target, mask_target, i_batch,
                   batch_size=batch_size):
    idx_from = i_batch * batch_size
    idx_to = (i_batch + 1) * batch_size

    X_batch = X_target[idx_from:idx_to].to(device)
    y_batch = y_target[idx_from:idx_to].to(device)
    mask_batch = mask_target[idx_from:idx_to].to(device)

    logits = model(X_batch, attention_mask=mask_batch)['logits']
    loss = F.cross_entropy(logits, y_batch)
    nrow = X_batch.shape[0]

    return loss, nrow

In [ ]:
for i_epoch in range(n_epoch):
    print(f'--- epoch {i_epoch + 1} / {n_epoch}')

    # 학습 단계
    loss_sum = 0
    model.train()

    for i_batch in range(n_batch_train):        
        optimizer.zero_grad()
        loss, nrow = calculate_loss(model, X_train, y_train, mask_train, i_batch)
        loss_sum += loss.cpu().detach().numpy().item() * nrow
        loss.backward()
        optimizer.step()

    loss_train = loss_sum / X_train.shape[0]
    print(f'  loss (train) : {loss_train:.4f}')

    # 검증 단계
    loss_sum = 0
    model.eval()

    for i_batch in range(n_batch_valid):
        loss, nrow = calculate_loss(model, X_valid, y_valid, mask_valid, i_batch)
        loss_sum += loss.cpu().detach().numpy().item() * nrow

    loss_valid = loss_sum / X_valid.shape[0]
    print(f'  loss (valid) : {loss_valid:.4f}')

## 8.4. LIME에 의한 모델 해석

### 확률 연산 함수의 준비

In [ ]:
def predict_proba(texts, model=model, tokenizer=tokenizer,
                  max_length=max_length, batch_size=batch_size):
    tokenized = tokenizer.batch_encode_plus(
        texts, padding=True, truncation=True,
        max_length=max_length, return_tensors='pt')
    ids, masks = [tokenized[key]
                  for key in ['input_ids', 'attention_mask']]    

    n_batch = ids.shape[0] // batch_size + 1    
    list_prob = []
    for i_batch in range(n_batch):
        idx_from = i_batch * batch_size
        idx_to = (i_batch + 1) * batch_size

        ids_batch = ids[idx_from:idx_to].to(device)
        mask_batch = masks[idx_from:idx_to].to(device)

        logits = model(ids_batch, attention_mask=mask_batch)['logits']
        prob = F.softmax(logits, dim=1).cpu().detach().numpy()
        list_prob.append(prob)

    return np.vstack(list_prob)

In [ ]:
word_tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name, do_subword_tokenize=False)

### LimeTextExplainer 의 생성

In [ ]:
explainer = LimeTextExplainer(
    class_names=target_categories,
    split_expression=word_tokenizer.tokenize,
    mask_string=tokenizer.pad_token,
    random_state=0)

### LIME 실행 예 1

In [ ]:
sample_text = '超薄型パソコンの新機種が年末までに世界での販売エリアを大幅拡大。'
exp_result = explainer.explain_instance(sample_text, predict_proba, num_features=5,
                                        labels=[target_categories.index('kaden-channel')])

In [ ]:
exp_result.show_in_notebook(text=True)

### LIME 실행 예 2

In [ ]:
sample_text = 'アカデミー賞受賞映画の今年のおすすめポイントを紹介します。'
exp_result = explainer.explain_instance(sample_text, predict_proba, num_features=5,
                                        labels=[target_categories.index('movie-enter')])

In [ ]:
exp_result.show_in_notebook(text=True)

## 8.5. Integrated Gradients 에 의한 방법

### 헬퍼 함수

In [ ]:
def encode_with_reference(tokenizer, text, device=device):
    encoded = tokenizer.encode_plus(text, return_tensors='pt')
    input_ids = encoded['input_ids']
    masks = encoded['attention_mask']
    ref_ids = torch.tensor([
        [tokenizer.cls_token_id] +
        [tokenizer.pad_token_id] * (input_ids.shape[1] - 2) +
        [tokenizer.sep_token_id],
    ])
    return input_ids.to(device), ref_ids.to(device), masks.to(device)

In [ ]:
def predict_func(input_ids, attention_mask=None, label=None,
                 model=model, tokenizer=tokenizer):
    logits = model(input_ids, attention_mask=attention_mask)['logits']
    if label is not None:
        result = F.softmax(logits, dim=1)[:, label]
    else:
        result = F.softmax(logits, dim=1)
    return result

- 샘플 텍스트로써 sports-watch 로 분류시키는 텍스트를 준비하였습니다.
- 「일본 대표 감독」과「감독을 퇴임」한다는 표현이 있으며,「감독」이라는 단어는 2번 사용되었습니다.

In [ ]:
sample_text = 'サッカー日本代表監督が、今日の敗戦結果を踏まえて監督を退任する意向を表明した。'
target_category = 'sports-watch'
target_label = target_categories.index(target_category)

In [ ]:
input_ids, ref_ids, masks = encode_with_reference(tokenizer, sample_text)
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

In [ ]:
prob = predict_func(input_ids, attention_mask=masks).cpu().detach().numpy().squeeze(0)
pred_label = np.argmax(prob)
pred_category = target_categories[pred_label]
pred_label_prob = prob[pred_label]
target_label_prob = prob[target_label]

### Integrated Gradient 실행

In [ ]:
lig = LayerIntegratedGradients(predict_func, model.bert.embeddings)
attributions, delta = lig.attribute(
    inputs=input_ids, baselines=ref_ids,
    additional_forward_args=(masks, target_label),
    return_convergence_delta=True)

In [ ]:
attributions_sum = attributions.sum(dim=-1).squeeze(0)
attributions_sum = attributions_sum / torch.norm(attributions_sum)

### 결과의 가시화

In [ ]:
classification_vis = viz.VisualizationDataRecord(
    attributions_sum,
    pred_label_prob,
    pred_category,
    target_category,
    '',
    attributions_sum.sum(),       
    tokens,
    delta)

In [ ]:
viz.visualize_text([classification_vis])

## 칼럼 : Attention 가시화
- bertviz 이라는 라이브러리에 의해 Attention 을 가시화 할 수 있습니다.

In [ ]:
#!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
    sys.path += ['bertviz_repo']

In [ ]:
from bertviz import head_view

In [ ]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min',
    jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

In [ ]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids'].to(device)
    if sentence_b:
        token_type_ids = inputs['token_type_ids'].to(device)
        attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        attention = model(input_ids)['attentions']
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start)

In [ ]:
sample_text = "アカデミー賞受賞映画の今年のおすすめポイントを紹介します"

In [ ]:
show_head_view(model, tokenizer, sample_text)